In [8]:
import logging
import numpy as np
from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve, accuracy_score, classification_report, confusion_matrix
from transformers import AutoImageProcessor, DefaultDataCollator
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer
from transformers import pipeline
from torchvision.transforms import ToPILImage
import matplotlib.pyplot as plt
import evaluate
import itertools
import sys
sys.path.append('../src')
import datafeed as feed

# Load Data

In [4]:
ls = feed.get_image_list()
#feed.collect_statistics(ls)
ds = feed.get_dataset_with_transform(ls, feed.transform_vit)

# Split Data

In [5]:
ds_split = feed.split_dataset_train_test_validate(ds)
ds_split

DatasetDict({
    validation: Dataset({
        features: ['label', 'image'],
        num_rows: 355
    })
    test: Dataset({
        features: ['label', 'image'],
        num_rows: 288
    })
    train: Dataset({
        features: ['label', 'image'],
        num_rows: 2902
    })
})

# Initialize Model

In [13]:
checkpoint = feed.CHECKPOINT_VIT_ONLINE
processor = feed.get_processor(checkpoint)

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(feed.id2label),
    id2label=feed.id2label,
    label2id=feed.label2id,
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Train Model

In [10]:
data_collator = DefaultDataCollator()
accuracy = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1) # highest prediction per label index
    return accuracy.compute(predictions=predictions, references=labels)

Change the following cell to fine-tune the parameters. Note the first line which defines the name of the directoy under which the results will be saved after training. Check if this directory name is valid; e.g. check if saving the results would accidently overwrite previous models with different parameters.

In [16]:
MODEL_SAVE_DIR = "vit-16"   # ViT Model with batch size 16

training_args = TrainingArguments(
    output_dir=f"./local_checkpoints/",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=ds_split["train"],
    eval_dataset=ds_split["test"],
    tokenizer=processor,
    compute_metrics=compute_metrics,
)


**This cell can take hours:**

In [ ]:

train_results=trainer.train()

# Save Model

In [ ]:
trainer.save_model(MODEL_SAVE_DIR)                   # Will save the model, so you can reload it using from_pretrained().
trainer.log_metrics("train", train_results.metrics)  
trainer.log_metrics("test", train_results.metrics)  
trainer.save_metrics("train", train_results.metrics) # save metrics into a .json file
trainer.save_state()

# Get Predictions for Evaluation Metrics

**Warning**: Untested code 

In [ ]:
classifier = pipeline("image-classification", model=model, image_processor=processor)

def get_pred_and_probablities(ds):
    """Return all predicitions and probabilities for DS."""
    images = []
    y_pred = []
    y_prob = []
    for row in ds:
        images += [ ToPILImage()(row['pixel_values']) ]  # Convert tensor to PIL Image
    for img in images:
        pred = classifier(img)  # returns an array of dicts {'label', 'score'}
        y_pred = feed.label2id[pred[0]['label']] # add first label as predicted label
        y_prob = pred[0]['score'] # add first score as predicted score

    return (np.array(y_pred), np.array(y_prob))

y_pred, y_prob = get_pred_and_probablities(ds_split['test'])
